In [1]:
import tornettools_util
import tornettools_plot
import subprocess

In [3]:
# args for tornettools parse; here using defaults

converge_time = 1200
run_time = -1

# parse performance stats only after the network has reached steady state
startts, stopts = converge_time, -1 if run_time < 0 else converge_time + run_time

## Extract download times (from the perspective of fileservers) from tgen data

In [4]:
def get_download_time(data, startts, stopts, bytekey):
    dt = {'ALL': []}

    # download times can differ by microseconds in tgen.
    # TODO: use
    # resolution = 1.0 / 1000000.0
    if 'data' in data:
        for name in data['data']:
            #if pattern.match(name) is None: 
            # pattern match not necessary to match since tgentools command already ensured only files matching pattern 'server[0-9]+exit\.tgen\.[0-9]+\.stdout' are parsed
            #    print('No match')
            #    continue
            db = data['data'][name]
            ss = db['tgen']['stream_summary']
            if bytekey in ss:
                for header in ss[bytekey]:
                    for secstr in ss[bytekey][header]:
                        sec = int(secstr) - 946684800
                        # print(f"Sec: {sec}")
                        if sec >= startts and (stopts < 0 or sec < stopts):
                            # print("sec criterion fulfilled")
                            #mydlcount += len(data['nodes'][name]['lastbyte'][header][secstr])
                            for dl in ss[bytekey][header][secstr]:
                                seconds = float(dl)
                                #item = [seconds, resolution]
                                item = seconds
                                dt['ALL'].append(item)
                                dt.setdefault(header, []).append(item)
    return dt

In [5]:
def extract_download_time(ind, variant, data, startts, stopts):
    key = "time_to_first_byte_recv"
    dt = get_download_time(data, startts, stopts, key)
    outpath = f"download_times/{ind}/{variant}.{key}.json"
    tornettools_util.dump_json_data(dt, outpath, compress=False)

    key = "time_to_last_byte_recv"
    dt = get_download_time(data, startts, stopts, key)
    outpath = f"download_times/{ind}/{variant}.{key}.json"
    tornettools_util.dump_json_data(dt, outpath, compress=False)

In [6]:
for variant in ["delay_neg_32", "delay_neg_32_zero", "tor_unchanged"]:

        for ind in [1012, 953, 1294, 599, 6938, 873]:

                # Uncomment below command to decompress relevant results
                # subprocess.run([f'unxz sim_res/{variant}/{ind}/tgen_servers. analysis.json.xz'], shell=True)

                data = tornettools_util.load_json_data(f'sim_res/{variant}/{ind}/tgen_servers.analysis.json')

                extract_download_time(ind, variant, data, startts, stopts)

## Plot full delay mechanism implementation vs. overhead only implementation as well as full delay mechanism implementation vs. unmodified Tor

In [2]:
def plot_download_time(args, tornet_dbs, bytes_key, versions_compared):
   
    # cache the corresponding data in the 'data' keyword for __plot_cdf_figure
    for tornet_db in tornet_dbs:
        tornet_db['data'] = [tornet_db['dataset'][i][bytes_key] for i, _ in enumerate(tornet_db['dataset']) if bytes_key in tornet_db['dataset'][i]]
    # for torperf_db in torperf_dbs:
    #    torperf_db['data'] = [torperf_db['dataset']['download_times']#[bytes_key]]

    # dbs_to_plot = torperf_dbs + tornet_dbs
    dbs_to_plot = tornet_dbs

    tornettools_plot.plot_cdf_figure(args, dbs_to_plot, f"{versions_compared}.transfer_time_{bytes_key}",
                      yscale="taillog",
                      xlabel=f"Transfer Time (s): Bytes={bytes_key}")

In [3]:
full = {
    'dataset': [tornettools_util.load_json_data(f'download_times/{ind}/delay_neg_32.time_to_last_byte_recv.json') for ind in [599, 953, 1012, 1294, 6938, 873]], 
    'label': "Full delay mechanism",
    'color': 'red',
}

overhead = {
    'dataset': [tornettools_util.load_json_data(f'download_times/{ind}/delay_neg_32_zero.time_to_last_byte_recv.json') for ind in [599, 953, 1012, 1294, 6938, 873]], 
    'label': "Overhead only",
    'color': 'yellow',
}

unmodified = {
    'dataset': [tornettools_util.load_json_data(f'download_times/{ind}/tor_unchanged.time_to_last_byte_recv.json') for ind in [599, 953, 1012, 1294, 6938, 873]], 
    'label': "Unmodified Tor",
    'color': 'yellow',
}


plot_download_time(None, [full, overhead], "51200", "full_vs_overhead")
plot_download_time(None, [full, overhead], "1048576", "full_vs_overhead")
plot_download_time(None, [full, overhead], "5242880", "full_vs_overhead")

plot_download_time(None, [full, unmodified], "51200", "full_vs_unmodified")
plot_download_time(None, [full, unmodified], "1048576", "full_vs_unmodified")
plot_download_time(None, [full, unmodified], "5242880", "full_vs_unmodified")